# Ex10 - Abrupt monsoon transitions

### Positive feedback systems in India's summer monsoon:

"The onset of monsoon precipitation in spring is initiated by the development of a tropospheric temperature contrast between land and ocean and associated convergence of moist air over the continent.
During the rainy season, the atmospheric heat budget in the monsoon region is then dominated by the so-called moisture–advection feedback:
<p>The release of latent heat by precipitation reinforces the land–ocean tropospheric temperature contrast, thereby stabilizing the circulation that brings in more moist air from the ocean, which in turn maintains precipitation. We argue that the moisture–advection feedback stabilizes the rainy monsoon regime by inducing persistence"</p>

This persistence leads to a hysterisis/"memory effect" that can be represented in a "day-to-day-model" where on any day $t$, the probability for rainfall $\tilde{p}_t$ is calculated as the normalization of percipitation parameters observed on $\tau$ previous days $P_t$:<br/>
- The temporal average of the percipitation $\langle{P}\rangle^{t-1}_{t-\tau}$
- The percipitation extremes $P_{-}$ and $P_{+}$

<div style="font-size: 20px"> $\tilde{p}_t \equiv \frac{\langle{P}\rangle^{t-1}_{t-\tau} - P_{-}}{P_{+} - P_{-}}$ </div>

In [25]:
import numpy as np

def daytoday_probability(P):
    """
    Returns the probability for rain calculated with the given list 
    of percipitation rates observed on previous days
    """ 
    return (np.average(P) - min(P)) / (max(p) - min(p))